# ch09 数据聚合与分组运算
本章主要学习：
+ 根据一个或多个键（可以是函数、数组或DataFrame列名）拆分pandas对象；
+ 计算分组摘要统计，如计数、平均值、标准差、或用户自定义函数；
+ 对DataFrame的列应用各种各样的函数；
+ 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等；
+ 计算透视表或交叉表；
+ 执行分位数分析以及其他分组分析；

## 9.1 GroupBy技术
分组键可以有多种形式，且类型不必相同：
+ 列表或数组，其长度与待分组的轴一样；
+ 表示DataFrame某个列名的值；
+ 字典或Series，给出待分组轴上的值与分组名之间的对应关系；
+ 函数，用于处理轴索引或索引中的各个标签

后三种都只是快捷方式而已，其最终目的就是产生一组用于拆分对象的值。

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                  'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),
                  'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.489158,-0.955641,a,one
1,-1.379867,0.016684,a,two
2,0.369426,-0.937115,b,one
3,1.041561,0.659981,b,two
4,-0.800869,-0.480562,a,one


In [3]:
# 按key1进行分组，并计算data1列的平均值
# 访问data1，并根据key1调用groupby
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a   -0.889964
b    0.705493
Name: data1, dtype: float64

In [4]:
# 一次传入多个数组
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.645013
      two    -1.379867
b     one     0.369426
      two     1.041561
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.645013,-1.379867
b,0.369426,1.041561


In [9]:
# 分组键可以是任何长度适当的数组
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -1.379867
            2006    0.369426
Ohio        2005    0.276202
            2006   -0.800869
Name: data1, dtype: float64

In [11]:
# 将列名（可以是字符串、数字或其他Python对象）用作分组键：
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.889964,-0.473173
b,0.705493,-0.138567


In [12]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.645013 -0.718101
     two  -1.379867  0.016684
b    one   0.369426 -0.937115
     two   1.041561  0.659981

In [13]:
# GroupBy的size方法，可以返回一个含有分组大小的Series
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 9.1.1 对分组进行迭代